In [1]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

In [2]:
x = np.load('data/EEG_all_epochs.npy')
y = np.load('data/EEG_contaminated_with_EOG/EEG_contaminated_with_EOG_1db.npy')

# Preprocesamiento de los datos en preparacion para la LSTM

In [3]:
#Normalizacion de los datos
scaler = StandardScaler()
x_norm = scaler.fit_transform(x)
y_norm = scaler.fit_transform(y)

#Reshape de los datos
timesteps = 100
n_samples, n_features = x_norm.shape
n_sequences = n_samples // timesteps
x_reshaped = np.reshape(x_norm[:n_sequences * timesteps, :], (n_sequences, timesteps, n_features))

n_samples, n_features = y_norm.shape
n_sequences = n_samples // timesteps
y_reshaped = np.reshape(y_norm[:n_sequences * timesteps, :], (n_sequences, timesteps, n_features))

print(x_reshaped.shape)
print(y_reshaped.shape)

(45, 100, 512)
(45, 100, 512)


In [4]:
x_train, x_test, y_train, y_test = train_test_split(x_reshaped, y_reshaped, test_size=0.2, random_state=42)

In [5]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(36, 100, 512)
(9, 100, 512)
(36, 100, 512)
(9, 100, 512)


In [6]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, input_shape=(timesteps, n_features)))
model.add(LSTM(32, return_sequences=True))
model.add(Dense(16, activation='relu'))
model.add(Dense(n_features))
model.compile(loss='mean_squared_error', optimizer='adam')


In [7]:
model.fit(x_train, y_train, batch_size=32, epochs=50, validation_data=(x_test, y_test))

Epoch 1/50
2/2 [==============================] - 8s 1s/step - loss: 0.9982 - val_loss: 1.0211
Epoch 2/50
2/2 [==============================] - 0s 98ms/step - loss: 0.9964 - val_loss: 1.0208
Epoch 3/50
2/2 [==============================] - 0s 99ms/step - loss: 0.9955 - val_loss: 1.0206
Epoch 4/50
2/2 [==============================] - 0s 99ms/step - loss: 0.9949 - val_loss: 1.0204
Epoch 5/50
2/2 [==============================] - 0s 99ms/step - loss: 0.9943 - val_loss: 1.0202
Epoch 6/50
2/2 [==============================] - 0s 100ms/step - loss: 0.9936 - val_loss: 1.0200
Epoch 7/50
2/2 [==============================] - 0s 101ms/step - loss: 0.9929 - val_loss: 1.0198
Epoch 8/50
2/2 [==============================] - 0s 102ms/step - loss: 0.9922 - val_loss: 1.0195
Epoch 9/50
2/2 [==============================] - 0s 100ms/step - loss: 0.9914 - val_loss: 1.0192
Epoch 10/50
2/2 [==============================] - 0s 100ms/step - loss: 0.9905 - val_loss: 1.0189
Epoch 11/50
2/2 [=========

In [8]:
score = model.evaluate(x_test, y_test)

1/1 [==============================] - 0s 34ms/step - loss: 0.9917


In [9]:
y_limpiado = model.predict(y_test)



1/1 [==============================] - 1s 1s/step


In [11]:
n_samples, n_timesteps, n_features = y_test.shape
y_test_reshaped = y_test.reshape((n_samples, n_features))

ValueError: cannot reshape array of size 460800 into shape (9,512)

In [ ]:
mse = mean_squared_error(y_limpiado, y_test)
r2 = r2_score(y_limpiado, y_test)

print("MSE:", mse)
print("R2:", r2)